In [1]:
import cudf 
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import roc_auc_score

import cupy

In [2]:
feat_cup = cudf.read_csv('/kaggle/input/ab-it-cup/all_features_final.csv', sep=';')

In [3]:
target = cudf.read_csv('/kaggle/input/ab-it-cup/owner_target_final.csv')

In [4]:
top_features = ['curbal_usd_amt_cm_avg',
 'curr_rur_amt_curr_v2',
 'age',
 'hdb_bki_total_active_products',
 'amount_by_category_90d__summarur_amt__SUM__cashflowcategory_name__vydacha_nalichnyh_v_bankomate',
 'transaction_category_supermarket_sum_amt_m2',
 'turn_save_db_sum_v2',
 'avg_amount_daily_transactions_30d',
 'hdb_bki_total_cnt',
 'ratio_amount_by_category_90d__summarur_amt__SUM__cashflowcategory_name__tovary_i_uslugi',
 'ratio_turn_save_db_sum_v2_total_inc',
 'stratsegfactor',
 'hdb_bki_total_max_limit',
 'transaction_category_supermarket_percent_cnt_2m',
 'curr_rur_amt_cm_avg',
 'avg_cnt_daily_transactions_30d',
 'hdb_bki_total_pil_last_days',
 'salary_2020',
 'diff_cur_cr_db_turn',
 'transaction_category_fastfood_inc_cnt_2m',
 'amount_by_category_90d__summarur_amt__SUM__cashflowcategory_name__taksi',
 'hdb_bki_total_products',
 'transaction_category_other_retail_purchase_inc_cnt_2m',
 'by_category__amount__SUM__eoperation_type_name__vneshnij_perevod_rur',
 'total_rur_amt_3m_avg',
 'avg_amount_daily_transactions_90d',
 'transaction_category_money_send_inc_cnt_2m',
 'profit_income_out_rur_amt_9m',
 'total_rur_amt_cm_avg',
 'hdb_bki_total_cc_cnt']

In [5]:
y = 1 - feat_cup.merge(target, how='left', left_on='client_id', right_on='owner_id')['owner_id'].isna().astype(int);

In [6]:
X = feat_cup[top_features]

In [7]:
cat_vals = X.columns[X.dtypes == object]
num_vals = X.columns[X.dtypes != object]

In [9]:
X[cat_vals] = X[cat_vals].fillna(X[cat_vals].mode()['stratsegfactor'][0])

In [10]:
X[num_vals] = X[num_vals].fillna(X[num_vals].min())

In [11]:
# from cuml.preprocessing import LabelEncoder

In [12]:
# X[cat_vals]['stratsegfactor']

In [13]:
# lb = LabelEncoder()
# cat_features = lb.fit_transform(X[cat_vals]['stratsegfactor'])

In [14]:
from sklearn.preprocessing import OneHotEncoder

In [15]:
one_hot = OneHotEncoder()
cat_features = one_hot.fit_transform(X.drop(num_vals, axis=1).to_pandas())

In [24]:
X = np.hstack([X.drop(cat_vals, axis=1).to_pandas(), cat_features.todense()])

In [25]:
X = cupy.array(X)
y = cupy.array(y)

In [202]:
from cuml.preprocessing import StandardScaler

In [203]:
scaler = StandardScaler()
train_X = scaler.fit_transform(X)

In [204]:
from cuml.linear_model import LogisticRegression

In [205]:
reg = LogisticRegression(max_iter=5000, C=0.001, penalty='l2', class_weight={0: 0.0001, 1: 100})

reg.fit(X,y)

[W] [15:39:35.264829] L-BFGS line search failed (code 4); stopping at the last valid step


LogisticRegression()

In [206]:
roc_auc_score(y_score=reg.predict_proba(X)[:, 1].get(), y_true=y.get())

0.4996863349707995

In [ ]:
X.shape

In [207]:
sgkf = StratifiedGroupKFold()
folds = sgkf.split(X.get(), y.get(), feat_cup['client_id'].to_pandas())

In [208]:
roc_auc_metrics = []
num_fold = 1
for (train_index, test_index) in folds:
    print(f'FOLD {num_fold}/5')
    scaler = StandardScaler()
    train_X = scaler.fit_transform(X[train_index])
    train_y = y[train_index]
    test_X = scaler.transform(X[test_index])
    test_y = y[test_index]
    
    
    reg = LogisticRegression(C=10, penalty='l1', class_weight='balanced')
    reg.fit(train_X, train_y)
    
    roc_auc_metrics.append(roc_auc_score(y_score=reg.predict_proba(test_X)[:, 1].get(), y_true=test_y.get()))
    num_fold += 1

FOLD 1/5
FOLD 2/5
FOLD 3/5
FOLD 4/5
FOLD 5/5


In [209]:
np.mean(roc_auc_metrics)

0.5067956193474407

In [154]:
np.mean(roc_auc_metrics)

0.5038632253262417

In [162]:
roc_auc_metrics

[0.5095117664319726,
 0.4996283532989414,
 0.5051201480671745,
 0.5014416413789571,
 0.5021127173598647]

In [26]:
from cuml.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X.astype('float32'), y)

roc_auc_score(y_score=clf.predict_proba(X.astype('float32'))[:, 1].get(), y_true=y.get())

0.9149395621462545

In [27]:
sgkf = StratifiedGroupKFold()
folds = sgkf.split(X.get(), y.get(), feat_cup['client_id'].to_pandas())

In [28]:
roc_auc_metrics = []
num_fold = 1
for (train_index, test_index) in folds:
    print(f'FOLD {num_fold}/5')
    train_X = X[train_index]
    train_y = y[train_index]
    test_X = X[test_index]
    test_y = y[test_index]
    
    
    clf = RandomForestClassifier()
    clf.fit(train_X.astype('float32'), train_y)
    
    roc_auc_metrics.append(roc_auc_score(y_score=clf.predict_proba(test_X.astype('float32'))[:, 1].get(), y_true=test_y.get()))
    num_fold += 1

FOLD 1/5
FOLD 2/5
FOLD 3/5
FOLD 4/5
FOLD 5/5


In [29]:
np.mean(roc_auc_metrics)

0.49399623097696593

In [30]:
roc_auc_metrics

[0.4932012687289128,
 0.49895979467575996,
 0.4913333857752421,
 0.49707275588311434,
 0.4894139498218003]